In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools
from itertools import combinations
import node2vec as Node2Vec
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from node2vec import Node2Vec
from sklearn.cluster import DBSCAN
import math

from networkx.algorithms.cycles import simple_cycles


#### Calculating Z-Score

In [ ]:
#-------calculating z_score for the modified formula--------#
def z_score_calc(value, mean_value, std_value):
    z_score = (value - mean_value) / std_value
    return z_score

#### Calculating Coefficient of variation for 3-variable

In [ ]:
#--------here, we have given 3 weight of the 3-cycle-------------#
def coeff_of_3_var(x, y, z):
    
    if(x==0 and y==0 and z==0): # show all weight is zero
        return 0
    if(x==y and y==z):          # all trader are transfering same amount
        return 1
    #-----------it show how much trader involve in circular trade--------------#
    #----------- it give the output in the form of percentage if it give 0.70, it's meaning is intensity of that it is circular trad is 70% means high---#
    a = x/y
    b = y/z
    c = z/x
    if(a>1): # outflow(money send) is less then inflow(money recieve) then set as 1
        a = 1
    if(b>1): # outflow is less then inflow then set as 1
        b=1
    if(c>1): # outflow is less then inflow then set as 1
        c=1
        
    return min(a,b,c)    
    

#### Calculating Coefficient of variation for 3-variable

In [ ]:
#--------here, we have given 2 weight of the 2-cycle-------------#
def coeff_of_2_var(x, y):
    
    if(x==0 and y==0):  # show all weight is zero
        return 0
    if(x==y):           # all trader are transfering same amount
        return 1
    a = x/y
    b = y/x
    if(a>1):   # outflow is less then inflow then set as 1
        a = 1
    if(b>1):   # outflow is less then inflow then set as 1
        b=1
            
    return min(a,b)    
    

### custom_weight function

In [ ]:
def custom_weight(node1, node2, mean_value, std_value):
   
    c2_w1 = 0    #first weight in cycle-2
    c2_w2 = 0    #second weight in cycle-2
    c3_w1 = 0    #first weight in cycle-3
    c3_w2 = 0    #second weight in cycle-3
    c3_w3 = 0    #third weight in cycle-3
    c2_count = 0  #count of cycle-2
    c3_count = 0  #count of cycle-3
    
    c2_count, c2_w1, c2_w2 = cycle_2_calc (node1, node2, cycles_2)  #gives number of 2 cycle and their two weight
    
    
    c3_count, max_cov, c3_w1, c3_w2, c3_w3 = cycle_3_calc(node1, node2, cycles_3)   #gives number of 3 cycle and their three weight
    
    
    c2_cov = np.exp(coeff_of_2_var(c2_w1, c2_w2))      #give the intensity of circular trading of 2-cycle
    c3_cov = np.exp(coeff_of_3_var(c3_w1, c3_w2, c3_w3))   #give the intensity of circular trading of 3-cycle
    
    value = c2_w1 + c2_w2 + c3_w1 + c3_w2 + c3_w3    
    
    z_score = z_score_calc(value, mean_value, std_value) # calculating z-score
    
    
    root_cyc = math.sqrt(pow(2, c2_count) * pow(3, c3_count))
    

    return root_cyc *c3_cov*z_score   # give the final weight of edge

#### calculating the total weights

In [ ]:
def total_weight_between_nodes(node1, node2):
    try:
        weight1 = G[node1][node2]['weight']
    except KeyError:
        weight1 = 0
    try:
        weight2 = G[node2][node1]['weight']
    except KeyError:
        weight2 = 0
    return weight1 + weight2


#### 2 - cycle calculation

In [ ]:
def cycle_2_calc(node1, node2, cycles_2):
    for cycle, weights in cycles_2:
        if set([node1, node2]) == set(cycle):
            return 1, weights[0], weights[1]
    return 0, 0, 0



In [ ]:
def cycle_3_calc(node1, node2, cycles_3):
    sum_w = 0
    max_cov = 0
    count = 0
    c3_w1 = 0
    c3_w2 = 0
    c3_w3 = 0
    
    for cycle, weights in cycles_3:
        if set([node1, node2]).issubset(set(cycle)):
            count+=1
            
            if((weights[0] + weights[1] + weights[2]) >= sum_w ):
                sum_w = weights[0] + weights[1] + weights[2]
                c3_w1 = weights[0]
                c3_w2 = weights[1]
                c3_w3 = weights[2]
            if(coeff_of_3_var(weights[0], weights[1], weights[2]) >= max_cov):
                max_cov = coeff_of_3_var(weights[0], weights[1], weights[2])
    return count, max_cov, c3_w1, c3_w2, c3_w3


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('flattened.csv')

# Calculate the mean and standard deviation of the 'Value' column
mean_value = df['Value'].mean()
std_value = df['Value'].std()

# Print the results
print('Mean value:', mean_value)
print('Standard deviation of value:', std_value)
                     

### Creating Directed Graph

In [ ]:
#-----creating directed graph----------#
G = nx.DiGraph()
for _, row in df.iterrows():
    G.add_edge(row['Seller ID'], row['Buyer ID'], weight=row['Value'])



In [ ]:
# -----------Initialize an empty list to store cycles of length 3------------#
cycles_3 = []

# ------------Loop over all simple cycles in the graph----------#
for cycle in nx.simple_cycles(G):
     # -----------Check if the cycle has length 3 -----------#
    if len(cycle) == 3:
        
        # -------Initialize an empty list to store the weights of the edges in the cycle-----#
        cycle_weights = []
        for i in range(2):
            # Try to get the weight of the edge connecting the current node to the next node in the cycle
            try:
                weight = G.get_edge_data(cycle[i], cycle[i+1])['weight']
                cycle_weights.append(weight)
            except KeyError:
                # Edge doesn't exist
                break
        else:  # This runs if the for loop completes without hitting a 'break'
            
            # Try to get the weight of the edge connecting the last node in the cycle to the first node
            try:
                weight = G.get_edge_data(cycle[2], cycle[0])['weight']
                cycle_weights.append(weight)
            except KeyError:
                # Edge doesn't exist
                break
            if len(cycle_weights) == 3:
                cycles_3.append((cycle, cycle_weights))



In [ ]:
# -----------Initialize an empty list to store cycles of length 2------------#
cycles_2 = []
visited = set()
# ------------Loop over all simple cycles in the graph----------#
for cycle in nx.simple_cycles(G):
    # -----------Check if the cycle has length 2 -----------#
    if len(cycle) == 2 and tuple(cycle) not in visited:
        visited.add(tuple(cycle))
        cycle_weights = []
        for i in range(1):
            # Try to get the weight of the edge connecting the current node to the next node in the cycle
            try:
                weight = G.get_edge_data(cycle[i], cycle[i+1])['weight']
                cycle_weights.append(weight)
            except KeyError:
                # Edge doesn't exist
                break
        else:  # This runs if the for loop completes without hitting a 'break'
            # Try to get the weight of the edge connecting the last node in the cycle to the first node
            try:
                weight = G.get_edge_data(cycle[1], cycle[0])['weight']
                cycle_weights.append(weight)
            except KeyError:
                # Edge doesn't exist
                break
            cycles_2.append((cycle, cycle_weights))




#### Mamoth block

In [ ]:
UG = nx.Graph()

# Add edges to the undirected graph
for u, v, data in G.edges(data=True):
    # Add the edge in both directions
    UG.add_edge(u, v)
    UG.add_edge(v, u)


In [ ]:
H = nx.Graph()
weight = 1
for u, v in UG.edges:
    print("u, v", u,v,custom_weight(u,v, mean_value, std_value))
    H.add_edge(u, v, weight=custom_weight(u,v, mean_value, std_value))

    
#######################################################################
nx.draw_networkx(H, with_labels=True)
plt.show()
#print(G.edges(data=True))

#### Embeddings + node2vec

In [ ]:
#applying node2vec on the graph H
node2vec = Node2Vec(H, dimensions=2, walk_length=30, num_walks=200)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Use the embeddings to cluster the nodes using DBSCAN
X = model.wv.vectors
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(X)
labels = dbscan.labels_

# Plot the scatter plot
plt.figure(figsize=(8,8))
plt.scatter(X[:,0], X[:,1], c=labels, cmap='viridis')
plt.show()

#### Printing the cluster

In [ ]:
clusters = {}
for i, label in enumerate(labels):
    if label in clusters:
        clusters[label].append(list(G.nodes())[i])
    else:
        clusters[label] = [list(G.nodes())[i]]

# Print the nodes in each cluster
for label, nodes in clusters.items():
    print("Nodes in cluster", label, ":", nodes)
